# NYC Automated Bicycle Counts
June 29, 2020
Alice Friedman

This code will download, summarize, and clean data collected in NYC via automatated bike counteres and available to the public via NYC Open Data portal. The count data and location data are from two seperate tables, which are joined in this code.

In [1]:
# make sure to install these packages before running:

import urllib.request, json, requests
import pandas as pd
from datetime import datetime
from sodapy import Socrata

## Method

Automated counter location names, ids, and other data are stored in a table available here.
 
 * https://data.cityofnewyork.us/Transportation/Bicycle-Counters/smn3-rzf9

For the purposes of this analysis we will only use the table to match location names to ids, which is the key in the bike count table. Other data, such as lat/long, is also available.

For locations with multiple counters or where multiple counters have been used over a period of years (e.g. Manhattan Bridge), a summary count (e.g. counts in both directions and for all periods counted) is stored in an id with `sens==0`.  The list of locations with these complete counts is then used to call to the API to download counts, which are collected in 15-minute increments, here:

* https://data.cityofnewyork.us/Transportation/Bicycle-Counts/uczf-rk3c

Counts are then cleaned to assign relevant data types (e.g. dates are stored as timestamps rather than text) and then summed by month.

Finally, partial year data (the first year any counter is available as well as the current year) is removed from teh data set.

### Locations table

In [2]:
#from open data
locations_url = 'https://data.cityofnewyork.us/resource/smn3-rzf9.csv'
locations_raw = pd.read_csv(locations_url)

In [18]:
#create & clean table of counter locations
locations = locations_raw[['name', 'id', 'sens', 'counter']]
locations = locations[locations['sens']==0] #includes just the sum of all counts at a location
locations = locations[~locations['name'].str.contains("Interference")] #selects out calibration counters
locations = locations[locations['counter'].notnull()] #selects only active counters
locations['id'] = locations['id'].astype(str)

#exclude 1st Ave (known to haev a lot of interference)
locations = locations[locations.name != '1st Avenue - 26th St N']

print(len(locations))
print(locations.dtypes)
locations

13
name       object
id         object
sens        int64
counter    object
dtype: object


,name,id,sens,counter
5,8th Ave at 50th St.,100057316,0,Y2H18055363
7,Kent Avenue Bike Path,100010019,0,Y2H13094302
10,Prospect Park West,100009425,0,Y2H13094304
14,Ed Koch Queensboro Bridge Shared Path,100009428,0,Y2H19111445
16,Columbus Ave at 86th St.,100057320,0,Y2H18055356
17,Manhattan Bridge Display Bike Counter,100047029,0,Y2H17062567
18,Staten Island Ferry,100010017,0,Y2H13094300
19,Manhattan Bridge Ped Path,100009426,0,Y2H13074107
20,Broadway at 50th St,100057318,0,Y2H18055362
21,Brooklyn Bridge Bike Path,100010022,0,Y2H13074106


### BIcycle Counts from API

In [19]:
#this section creates and runs a function to page through the OpenDate API to get all counts for the 
#locations in the table above


client = Socrata("data.cityofnewyork.us", None) #none refers to token -- none required for public data


#functon to page through data and load data based on id
#returns a data frame  containing all available counts for given id

def load_data(loc_id):
    l = [] #empty dataframe

    n=0 #set counter
    loc = 'id=' + str(loc_id)
    lim=500000 #limit on API

    while True:
    # First 500000 results (max), returned as JSON from API / converted to Python list of
    # dictionaries by sodapy.
        results = client.get("uczf-rk3c", limit=lim, offset=lim*n, where=loc)
        frame = pd.DataFrame.from_records(results)
        #print(frame[0:1])
        l.append(frame)
        #print("n="+str(n))
        #print("length of l="+str(len(l)))
        n = n + 1
        if len(frame)<1:
            break
    
    return (pd.concat(l))

# use function to create list of dataframes for each id

dataList = []
for loc_id in locations['id']:
    print("loading data for location " + str(loc_id))
    dataList.append(load_data(loc_id))  

loading data for location 100057316
loading data for location 100010019
loading data for location 100009425
loading data for location 100009428
loading data for location 100057320
loading data for location 100047029
loading data for location 100010017
loading data for location 100009426
loading data for location 100057318
loading data for location 100010022
loading data for location 100010018
loading data for location 100057319
loading data for location 100009427


### Filter data prior to calibration

Certain locations have experienced known electircal intereference and were manually calibrated on a certain date. This data is located in the data dictionary for [Bicycle Counts on Open Data](https://data.cityofnewyork.us/Transportation/Bicycle-Counts/uczf-rk3c). I have manually created a table of this data which is linked on my [Bicycle Counters Repository on GitHub](https://raw.githubusercontent.com/aliceafriedman/BikeCounters).

In [20]:
#load interference dates (mannually entered as CSV from metadata in Open Data)
#pull from GitHub
#store as dict
calibration_date_raw = pd.read_csv('https://raw.githubusercontent.com/aliceafriedman/BikeCounters/master/FilteredLoc.csv')

#table of dates for locations with known calibration starts
calibration_date = pd.DataFrame(calibration_date_raw.dropna())
#calibration_date['id'] = calibration_date['id'].astype(str)

c_date = pd.to_datetime(calibration_date['filterBefore'], infer_datetime_format=True)

c_dict = dict(zip(calibration_date['id'], c_date))

print(c_dict)

{100010020: Timestamp('2016-11-01 00:00:00'), 100057320: Timestamp('2019-12-05 00:00:00'), 100047029: Timestamp('2018-08-23 00:00:00'), 100057318: Timestamp('2019-12-05 00:00:00'), 100057319: Timestamp('2019-12-05 00:00:00'), 100057316: Timestamp('2019-12-05 00:00:00'), 100010019: Timestamp('2016-12-13 00:00:00')}


### Additional data cleaning

Data is further cleaned to correct data types and select relevant fields. The table `counts` is a cleaned version of the 14 locations in 15-minute increments to which a 'day_of_week' column has been added.

In [21]:
# filters out data before calibration date, if applicable, before concatenating data from each location
#doing this with a list because different locations have different filterBefore dates
filtered_counts = []
for i in range(len(dataList)):
    k = dataList[i]['id'][0]
    if k in c_dict:
        f_date = c_dict[k]
        dataList[i]['date'] = pd.to_datetime(dataList[i]['date'], infer_datetime_format=True)
        cond = dataList[i]['date'] > f_date
        filtered_counts.append(dataList[i][cond])
        #dataList[i] = dataList[i][]
    else:
        filtered_counts.append(dataList[i])

In [22]:
counts = pd.concat(filtered_counts)

#correct data types
counts['counts'] = counts['counts'].astype(int)
counts['datetime'] = pd.to_datetime(counts['date'], infer_datetime_format=True)
counts['date'] = counts['datetime']
counts = counts.set_index(pd.DatetimeIndex(counts['datetime']))

#drop unwanted columns
counts = counts.drop(['datetime', 'status'], axis=1)

#add day_of_week 0 = Monday 6 = Sunday
counts['day_of_week'] = counts['date'].dt.weekday

isweekday = []
for day in counts['day_of_week']:
    if day < 5:
        isweekday.append(True)
    else: 
        isweekday.append(False)
counts['isweekday'] = isweekday 


print(counts.index)
print(counts.dtypes)
counts.head()

DatetimeIndex(['2020-06-14 01:00:00', '2020-06-14 00:45:00',
               '2020-06-14 00:30:00', '2020-06-14 00:15:00',
               '2020-06-14 00:00:00', '2020-06-13 23:45:00',
               '2020-06-13 23:30:00', '2020-06-13 23:15:00',
               '2020-06-13 23:00:00', '2020-06-13 22:45:00',
               ...
               '2020-03-15 17:30:00', '2020-03-15 17:45:00',
               '2020-03-15 18:00:00', '2020-03-15 18:15:00',
               '2020-03-15 18:30:00', '2020-03-15 18:45:00',
               '2020-03-15 19:00:00', '2020-03-15 19:15:00',
               '2020-03-15 19:30:00', '2020-03-15 19:45:00'],
              dtype='datetime64[ns]', name='datetime', length=1705325, freq=None)
counts                  int64
date           datetime64[ns]
id                     object
day_of_week             int64
isweekday                bool
dtype: object


,counts,date,id,day_of_week,isweekday
datetime,,,,,
2020-06-14 01:00:00,1,2020-06-14 01:00:00,100057316,6,False
2020-06-14 00:45:00,3,2020-06-14 00:45:00,100057316,6,False
2020-06-14 00:30:00,0,2020-06-14 00:30:00,100057316,6,False
2020-06-14 00:15:00,0,2020-06-14 00:15:00,100057316,6,False
2020-06-14 00:00:00,4,2020-06-14 00:00:00,100057316,6,False


In [23]:
counts.head()

,counts,date,id,day_of_week,isweekday
datetime,,,,,
2020-06-14 01:00:00,1,2020-06-14 01:00:00,100057316,6,False
2020-06-14 00:45:00,3,2020-06-14 00:45:00,100057316,6,False
2020-06-14 00:30:00,0,2020-06-14 00:30:00,100057316,6,False
2020-06-14 00:15:00,0,2020-06-14 00:15:00,100057316,6,False
2020-06-14 00:00:00,4,2020-06-14 00:00:00,100057316,6,False


### Sum by month

In [24]:
#resample and sum by month
m_counts_total = counts[['id', 'counts']].groupby('id').resample('M').sum().reset_index()

index = pd.MultiIndex.from_tuples(zip(m_counts_total['id'], m_counts_total['datetime']))
m_counts_total = m_counts_total.set_index(index)

print(m_counts_total.dtypes)
m_counts_total.head()

id                  object
datetime    datetime64[ns]
counts               int64
dtype: object


id   datetime  counts
100009425 2016-11-30  100009425 2016-11-30   38272
          2016-12-31  100009425 2016-12-31   35955
          2017-01-31  100009425 2017-01-31   32039
          2017-02-28  100009425 2017-02-28   36430
          2017-03-31  100009425 2017-03-31   35263

### Sum weekend & weekday by month

In [25]:
wkend_counts = counts[counts['isweekday'] == False]
wday_counts = counts[counts['isweekday'] == True]

m_counts_wkend = wkend_counts[['id', 'counts']].rename(columns={'counts':'weekend_counts'}).groupby('id').resample('M').sum()
m_counts_wday = wday_counts[['id', 'counts']].rename(columns={'counts':'weekday_counts'}).groupby('id').resample('M').sum()

monthly_counts = pd.concat([m_counts_total, m_counts_wkend, m_counts_wday], axis=1, join='inner')
monthly_counts.head()

,,id,datetime,counts,weekend_counts,weekday_counts
100010022,2014-11-30,100010022,2014-11-30,0,0,0
100009425,2019-01-31,100009425,2019-01-31,41954,10425,31529
100010017,2019-12-31,100010017,2019-12-31,7096,1801,5295
100010019,2018-02-28,100010019,2018-02-28,36375,6281,30094
100057320,2020-01-31,100057320,2020-01-31,37888,8433,29455


### Join to `locations` to add location name

In [26]:
locs = locations.set_index(locations['id'])
locs = locs[['name']]
m_counts = monthly_counts.set_index('id')
monthly_counts_named = pd.concat([m_counts, locs], axis=1, join='inner')

print(monthly_counts_named.dtypes)
monthly_counts_named.head()

datetime          datetime64[ns]
counts                     int64
weekend_counts             int64
weekday_counts             int64
name                      object
dtype: object


,datetime,counts,weekend_counts,weekday_counts,name
id,,,,,
100010022,2014-11-30,0,0,0,Brooklyn Bridge Bike Path
100009425,2019-01-31,41954,10425,31529,Prospect Park West
100010017,2019-12-31,7096,1801,5295,Staten Island Ferry
100010019,2018-02-28,36375,6281,30094,Kent Avenue Bike Path
100057320,2020-01-31,37888,8433,29455,Columbus Ave at 86th St.


## Check data for missing values

Some months are 0?

In [41]:
missing_months = monthly_counts_named[monthly_counts_named['counts']==0].sort_values(by=['datetime'])
print(missing_months)

            datetime  counts  weekend_counts  weekday_counts  \
id                                                             
100010022 2014-10-31       0               0               0   
100010022 2014-11-30       0               0               0   
100057316 2018-06-30       0               0               0   
100057316 2018-07-31       0               0               0   
100057316 2018-08-31       0               0               0   
100057316 2018-09-30       0               0               0   
100057316 2018-10-31       0               0               0   
100057316 2018-11-30       0               0               0   
100057316 2018-12-31       0               0               0   
100057316 2019-01-31       0               0               0   
100057316 2019-02-28       0               0               0   
100057316 2019-03-31       0               0               0   
100057316 2019-04-30       0               0               0   
100057316 2019-05-31       0            

# Remove partial years of data
This section removes partial years of data by removing the first (always partial) year of data for each location as well as the current year.

There are counts missing for part of 2014 at Brooklyn Bridge and 0s for some of 8th Ave... not sure what the story is there, but removing 8th Ave 0's results in no full years of data for that location.


Note: This code would remove a row in error if any location started counting in January, but that turns out not to be the case.

In [32]:
#remove 0 counts. 
has_counts = monthly_counts_named[monthly_counts_named['counts'] != 0]

def remove_first_yr_and_current_yr(df):
    print (str(len(df)) + " rows in initial data")
    allNames = df['name'].unique() #list of unique names in DF
    l = [] #empty list
    i = 0 #set counter
    for name in allNames:
        l.append(df[df['name']==name]) #seperate dataframe into list by name
        data = l[i]
        first_year = data['datetime'].min().year #stores first year of data for each location
        
        #condition
        remove_first_yr = data['datetime'].dt.year > first_year
        remove_current_year = data['datetime'].dt.year < datetime.today().year
        
        #filter each dataframe for conditions
        l[i] = data[remove_first_yr & remove_current_year]        
        #print("removing partial data from " + name + " for year " + str(first_year))
    
        i += 1 #increment counter
    
    result = pd.concat(l) # recombine filtered lists into df
    print(str(len(result)) + " rows returned")
    
    result = result.sort_values(by=['datetime']) #sort ascending
    
    return result # returns dataframe


full_yr_monthly_counts = remove_first_yr_and_current_yr(has_counts) 

print(full_yr_monthly_counts.head())
print(full_yr_monthly_counts.tail())

554 rows in initial data
430 rows returned
            datetime  counts  weekend_counts  weekday_counts  \
id                                                             
100010022 2014-01-31   14579            2033           12546   
100009427 2014-01-31   41692            7292           34400   
100009428 2014-01-31   27048            4648           22400   
100009426 2014-01-31    1215             271             944   
100009428 2014-02-28   22048            7133           14915   

                                            name  
id                                                
100010022              Brooklyn Bridge Bike Path  
100009427          Williamsburg Bridge Bike Path  
100009428  Ed Koch Queensboro Bridge Shared Path  
100009426              Manhattan Bridge Ped Path  
100009428  Ed Koch Queensboro Bridge Shared Path  
            datetime  counts  weekend_counts  weekday_counts  \
id                                                             
100010019 2019-12-31   

## Conclusions
The table below includes bicycle counts for all locations for which there is an active bike counter in NYC, including a monthly total, monthly total of weekdays, and monthly total of weekend days.

Known issues:

* Missing data for November and December of 2014 at Brooklyn Bridge

In [42]:
#write table
full_yr_monthly_counts.to_csv("full_yr_monthly_counts_clean.csv")